# Cat vs Non-Cat Classifier (Synced)

This notebook runs the code from `cat_classifier.py` so it stays in sync with the script.

- Data loading and preprocessing
- Training with regularization, optimizers, and mini-batches
- Metrics and visualization utilities
- Experiment comparisons (init, normalization, LR, LR decay)

In [ ]:
from pathlib import Path
import sys


def find_repo_file(filename, max_levels=6):
    current = Path.cwd().resolve()
    for _ in range(max_levels + 1):
        candidate = current / filename
        if candidate.exists():
            return candidate
        current = current.parent
    raise FileNotFoundError(f"Could not find {filename} from {Path.cwd()}")


script_path = find_repo_file("cat_classifier.py")
# Ensure imports resolve from the script's directory.
sys.path.insert(0, str(script_path.parent))
print(f"Loading: {script_path}")
code = script_path.read_text()
exec(code, globals())